Offset path generator

In [ ]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc
from svgpathtools import parse_path, Line, Path, wsvg

# Sol here https://github.com/mathandy/svgpathtools/blob/fcb648b9bb9591d925876d3b51649fa175b40524/svgpathtools/path.py#L710
def find_intersection(path):
    intersections = []
    for ids, seg in enumerate(path[:-2]): #-2 because next comment
        for ido, other_seg in enumerate(path[ids+2:]): #+2 to ignore the inmediate segment 
            if seg.intersect(other_seg) and ids != 0 and ids+ido+1!=len(path)-2:
                intersections.append((ids, ids + ido + 1))

    print(intersections)            
    return intersections

def del_intersections(path, intersections):
    sorted_indexes = sorted(intersections, key=lambda x: x[0], reverse=True)
    for start, end in sorted_indexes:
        path[start:end+1] = [Line(path[start-1].end,path[end+2].start)]


def offset_curve(path, offset_distance, steps=1000):
    """Takes in a Path object, `path`, and a distance,
    `offset_distance`, and outputs an piecewise-linear approximation 
    of the 'parallel' offset curve."""
    offset_segments = []

    segments = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
    segments.append((path[-1],path[0]))
    # print(*segments,sep='\nxd\n')

    for idx, (seg, next_seg) in enumerate(segments):
        normal_delta = abs(seg.unit_tangent(1)-next_seg.unit_tangent(0))

        nls = []
        for k in range(steps):
            t = k / steps
            offset_vector = offset_distance * seg.normal(t)
            nl = Line(seg.point(t), seg.point(t) + offset_vector)
            nls.append(nl)
        connect_the_dots = [Line(nls[k].end, nls[k+1].end) for k in range(len(nls)-1)]
        
        intersections = find_intersection(connect_the_dots)
        del_intersections(connect_the_dots,intersections)
        
        # if intersections:
        #     for intersection in intersections:
        #         cut_start,cut_end = intersection
        #         connect_the_dots[cut_start:cut_end+1] = [Line(connect_the_dots[cut_start-1].end, connect_the_dots[cut_end+2].start)]

        offset_segments.extend(connect_the_dots)

        # fix pointy joints #implement determination of large_arc and sweep values  
        if normal_delta > 0.1 and (idx!=len(segments)-1 or path.isclosed()):
            start = seg.end + offset_distance * seg.normal(1)
            end = next_seg.start + offset_distance * next_seg.normal(0)
            radius = offset_distance + offset_distance*1j
            rotation = 0
            large_arc = False
            sweep = False
            arc = Arc(start=start, radius=radius, rotation=rotation, large_arc=large_arc, sweep=sweep, end=end)
            offset_segments.extend([Line(offset_segments[-1].end,arc.start),arc])

    intersections = find_intersection(offset_segments)
    del_intersections(offset_segments,intersections)
    # print(intersections)
        # intersections = find_intersection(offset_segments)
        # if intersections:
        #     for intersection in intersections:
        #         cut_start,cut_end = intersection
        #         offset_segments[cut_start:cut_end+1] = [Line(offset_segments[cut_start-1].end, offset_segments[cut_end+2].start)]


    if path.isclosed():
        connect_the_dots.append(Line(nls[-1].end, nls[0].end))
    
    offset_path = Path(*offset_segments)
    return offset_path

# Examples:
seg1 = CubicBezier(230+200j, 100+100j, 200+200j, 200+300j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = Line(200+300j, 150+300j)  # A line beginning at (200, 300) and ending at (250, 350)
paths = Path(seg1,seg2)

distance = 10
offset_paths = offset_curve(paths, distance)

# Let's take a look
wsvg([paths, offset_paths], 'g' + 'r', filename='offset_curves.svg')
beep()

In [ ]:
!code offset_curves.svg

In [ ]:
!code offset_curves.svg

Generate Character Paths

In [ ]:
import matplotlib.pyplot as plt
from svgpathtools import svg2paths, svg2paths2, wsvg

def char2paths(char): # Implement character with multiple path, maybe ussing parsing

    plt.figure(figsize=(4, 4))
    plt.text(0.5, 0.5, char[0], fontsize=200, ha='center', va='center')
    plt.axis('off')
    filename='.matplot'
    plt.savefig(filename, format='svg')
    plt.close()
    
    paths, _, _ = svg2paths2(filename)
    paths.pop(0)
    wsvg(paths, filename='svgtools.svg')
    
    return paths

In [ ]:
paths = char2paths('T')
distance = -100
# print(*paths)
offset_paths = offset_curve(*paths, distance)
# Let's take a look
wsvg([*paths, offset_paths], 'g' + 'r', filename='offset_curves.svg')
beep()

In [ ]:
!code svgtools.svg

In [ ]:
!code offset_curves.svg

In [ ]:
import numpy as np
import simpleaudio as sa

def generate_tone(frequency, duration, sample_rate=44100):
    # Generar el tiempo en segundos para el arreglo de muestras
    t = np.linspace(0, duration, int(sample_rate * duration), False)

    # Generar el tono utilizando la función sinusoidal
    tone = np.sin(frequency * 2 * np.pi * t)

    # Asegurarse de que el arreglo esté en el rango [-1, 1]
    tone *= 0.5

    return tone

def beep(sample_rate=44100):
    # Convertir los valores de la señal a int16 (16-bit signed integer)
    frequency = 440  # Frecuencia en Hz (A4 en el piano)
    duration = 0.08   # Duración en segundos
    tone = generate_tone(frequency, duration)

    samples = (tone * 32767).astype(np.int16)

    # Crear el objeto de audio para reproducir la señal
    playback_obj = sa.play_buffer(samples, num_channels=1, bytes_per_sample=2, sample_rate=sample_rate)

    # Esperar hasta que termine la reproducción
    playback_obj.wait_done()

In [ ]:
import warnings

def self_intersect(path, other_curve, justonemode=False, tol=1e-12):
    """Finds intersections of `self` with `other_curve`

    Args:
        other_curve: the path or path segment to check for intersections
            with `self`
        justonemode (bool): if true, returns only the first
            intersection found.
        tol (float): A tolerance used to check for redundant intersections
            (see comment above the code block where tol is used).

    Returns:
        (list[tuple[float, Curve, float]]): list of intersections, each
            in the format ((T1, seg1, t1), (T2, seg2, t2)), where
            self.point(T1) == seg1.point(t1) == seg2.point(t2) == other_curve.point(T2)

    Scope:
        If the two path objects coincide for more than a finite set of
        points, this code will iterate to max depth and/or raise an error.
    """
    path1 = path
    path2 = other_curve if isinstance(other_curve, Path) else Path(other_curve)

    # assert path1 != path2
    # warnings.warn('same path detected')

    intersection_list = []
    for seg1 in path1:
        for seg2 in path2:
            if justonemode and intersection_list:
                return intersection_list[0]

            if seg1 != seg2:
                for t1, t2 in seg1.intersect(seg2, tol=tol):
                    T1 = path1.t2T(seg1, t1)
                    T2 = path2.t2T(seg2, t2)
                    intersection_list.append(((T1, seg1, t1), (T2, seg2, t2)))

    if justonemode and intersection_list:
        return intersection_list[0]

    # Note: If the intersection takes place at a joint (point one seg ends
    # and next begins in path) then intersection_list may contain a
    # redundant intersection.  This code block checks for and removes said
    # redundancies.
    if intersection_list:
        pts = [_seg1.point(_t1) for _T1, _seg1, _t1 in list(zip(*intersection_list))[0]]
        indices2remove = []
        for ind1 in range(len(pts)):
            for ind2 in range(ind1 + 1, len(pts)):
                if abs(pts[ind1] - pts[ind2]) < tol:
                    # then there's a redundancy. Remove it.
                    indices2remove.append(ind2)
        intersection_list = [inter for ind, inter in
                                enumerate(intersection_list) if
                                ind not in indices2remove]
    return intersection_list

In [ ]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc
from svgpathtools import disvg
from svgpathtools import kinks, smoothed_joint, smoothed_path

seg1 = CubicBezier(300+100j, 100+100j, 200+200j, 200+300j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = Line(200+300j, 250+350j)  # A line beginning at (200, 300) and ending at (250, 350)
path = Path(seg1, seg2)  # A path traversing the cubic and then the line
path.append(CubicBezier(250+350j, 275+350j, 250+225j, 200+100j))
# print(kinks(path))

intersections=[]
for (T1, seg1, t1), (T2, seg2, t2) in self_intersect(path,path):
        intersections.append(path.point(T1))
# print(intersections)
disvg(path, filename='intersections.svg',openinbrowser=False)
# disvg(path,nodes=intersections, filename='intersections.svg',openinbrowser=False)

In [ ]:
intersections=[]
for (T1, seg1, t1), (T2, seg2, t2) in self_intersect(path,path,tol=1e-26):
        intersections.append(path.point(T1))
print(intersections)

# self_intersect(path,path)